# Setup

In [149]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics
import autosklearn.regression
import matplotlib.pyplot as plt
import pickle

# Dataset

In [150]:
# Load database, Metadata and mapping tables
df = pd.read_csv("complete_ppjn_db.csv")
df_category = pd.read_csv("category.csv")
df_airline = pd.read_csv("metadata_ctry_airlines.csv")
df_income_code_cur = pd.read_csv("metadata_ctry_salary_2.csv")
df_index = pd.read_csv("cost_of_living_index.csv")
df_inflation = pd.read_csv("inflation_amended.csv")
df_XR = pd.read_csv("XR_list.csv")
df_tax = pd.read_csv("income_tax_updated.csv")

# Cleaning and mapping

## Cleaning

In [151]:
# Create a new column based on the conditions you specified
df["status"] = 1  # Default value is 1
df.loc[(df["Year"] == 2023) & (df["gross-net"] == "gross"), "status"] = 0  # If year=2023 and gross-net=gross, set to 0
df.loc[df["salary"].isnull() | (df["salary"] == "") | df["salary"].isna(), "status"] = 2  # If salary is blank, set to 2

# Apply transformations to selected rows
df.loc[df["status"] != 2, "gross-net"] = df["gross-net"].fillna("gross")
df.loc[df["status"] != 2, "salary"] = df["salary"].str.replace(',', '.')
df.loc[df["status"] != 2, "salary"] = pd.to_numeric(df["salary"])
df["Year"] = df["Year"].fillna(0)
df.loc[df["status"] != 2, "Year"] = df["Year"].astype(int)

# Remove rows where year > 2024
df = df.drop(df[df["Year"] > 2024].index)


In [152]:
df["Year"] = df["Year"].astype(int)
df

,URL,Airline,Year,Top/Base,Salary_info,salary,currency,gross-net,status
0,https://www.pilotjobsnetwork.com/jobs/British_...,British Airways Mainline,2023,CaptMax,"Short Haul Â£166,517 Long Haul Â£196,438",181477.5,GBP,gross,0
1,https://www.pilotjobsnetwork.com/jobs/British_...,British Airways Mainline,2023,CaptMin,"Short Haul & Long Haul Â£87,876 Based on year ...",87876.0,GBP,gross,0
2,https://www.pilotjobsnetwork.com/jobs/Cargo_Air,Cargo Air,2019,CaptMax,Basic salary: 4000 EUR/month (after taxes). Pe...,48000.0,EUR,net,1
3,https://www.pilotjobsnetwork.com/jobs/Cargo_Air,Cargo Air,2019,CaptMin,Basic salary: 3500 EUR/month (after taxes). Pe...,42000.0,EUR,net,1
4,https://www.pilotjobsnetwork.com/jobs/DHL_Air_UK,DHL Air UK,2023,CaptMax,172102,172102.0,GBP,gross,0
...,...,...,...,...,...,...,...,...,...
993,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,FlyinGroup,0,CaptMin,NaN,NaN,NaN,NaN,2
994,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,MS AVIATION,0,CaptMax,NaN,NaN,NaN,NaN,2
995,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,MS AVIATION,0,CaptMin,NaN,NaN,NaN,NaN,2
996,https://www.pilotjobsnetwork.com/jobs/ESMA_Avi...,ESMA Aviation,0,CaptMax,NaN,NaN,NaN,NaN,2


## Mapping

In [153]:
#Create df with all features
df_conso = df[["Airline", "Year", "Top/Base", "salary", "currency", "gross-net", "status"]]

#Add category
cat_map = dict(zip(df_category['Airline'], df_category['Category']))
df_conso["Category"] = df_conso["Airline"].map(cat_map)

#Add country
country_map = dict(zip(df_airline['Airline'], df_airline['ISO']))
df_conso["Country"] = df_conso["Airline"].map(country_map)

#Add income
income_map = dict(zip(df_income_code_cur["ISO"], df_income_code_cur["yearly_income"]))
df_conso["Income"] = df_conso["Country"].map(income_map)

#Add cost of living
index_map = dict(zip(df_index["ISO"], df_index["Cost of Living Index"]))
df_conso["Index"] = df_conso["Country"].map(index_map)

/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_18102/1868061681.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conso["Category"] = df_conso["Airline"].map(cat_map)


In [154]:
df_conso_2 = df_conso

# Update salary for 0 and 1

In [155]:
mask = (df_conso["status"] == 0) | (df_conso["status"] == 1)
if mask.any():
    df_conso = df_conso.dropna()
    for x in range(2010, 2023):
        df_inflation[f"{x}"] = df_inflation[f"{x}"].str.replace(',', '.').astype(float)
        df_inflation.fillna(0, inplace=True)
        df_inflation[f"{x}"] = df_inflation[f"{x}"].replace(np.nan,0)
        df_inflation[f"{x}"] = df_inflation[f"{x}"]/100+1
        df_inflation["2023"] = 1
    # df_inflation = df_inflation.set_index("ISO_name")

    def update_year(year):
        if year < 2010:
            return 2010
        else:
            return year

    df_conso['Year'] = df_conso['Year'].apply(update_year)

    df_inflation["Cum2010"]= df_inflation["2011"]*df_inflation["2012"]*df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2011"]= df_inflation["2012"]*df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2012"]= df_inflation["2013"]*df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2013"]= df_inflation["2014"]*df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2014"]= df_inflation["2015"]*df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2015"]= df_inflation["2016"]*df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2016"]= df_inflation["2017"]*df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2017"]= df_inflation["2018"]*df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2018"]= df_inflation["2019"]*df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2019"]= df_inflation["2020"]*df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2020"]= df_inflation["2021"]*df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2021"]= df_inflation["2022"]*df_inflation["2023"]
    df_inflation["Cum2022"]= df_inflation["2023"]
    df_inflation["Cum2023"]= 1

    # Standardize the salary in USD/GROSS/YEARLY/EST.2023

    #rate
    xr_dict = dict(zip(df_XR.Currency, df_XR.XR))
    df_conso['new_salary_USD'] = df_conso.apply(lambda x: x['salary'] / xr_dict[x['currency']] if x['currency'] != 'USD' else x['salary'], axis=1)
    #gross
    tax_dict = dict(zip(df_tax.ISO, df_tax.Income_tax))
    df_conso['new_salary_USD_GROSS'] = df_conso.apply(lambda x: x['new_salary_USD'] * (1+tax_dict[x['Country']]/100) if x['gross-net'] == 'net' else x['new_salary_USD'], axis=1)
    # inflation
    def calculate_new_salary(row):
        country = row['Country']
        year = row['Year']
        inflation_rate = df_inflation.loc[df_inflation["ISO_name"] == country, f"Cum{year}"].values
        if len(inflation_rate) == 0:
            # handle case where inflation rate is not found
            return None
        else:
            return row['new_salary_USD_GROSS'] * inflation_rate[0]

    df_conso["new_salary_USD_GROSS_2023"] = df_conso.apply(calculate_new_salary, axis=1)

/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_18102/2579442488.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conso['Year'] = df_conso['Year'].apply(update_year)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykernel_18102/2579442488.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conso['new_salary_USD'] = df_conso.apply(lambda x: x['salary'] / xr_dict[x['currency']] if x['currency'] != 'USD' else x['salary'], axis=1)
/var/folders/y9/x007ynb92t57yt7s517ct53w0000gn/T/ipykern

In [156]:
df_model = df_conso[["Airline", "Top/Base", "Category", "Country", "Income", "Index", "new_salary_USD_GROSS_2023", "status"]]

In [157]:
df_model

,Airline,Top/Base,Category,Country,Income,Index,new_salary_USD_GROSS_2023,status
0,British Airways Mainline,CaptMax,Legacy,United Kingdom,44480.0,61.5,218679.206632,0
1,British Airways Mainline,CaptMin,Legacy,United Kingdom,44480.0,61.5,105890.008194,0
4,DHL Air UK,CaptMax,Cargo,United Kingdom,44480.0,61.5,207381.790138,0
5,DHL Air UK,CaptMin,Cargo,United Kingdom,44480.0,61.5,146582.638454,0
6,LOT Polish Airlines,CaptMax,Major/National/Low Cost,Poland,16850.0,38.6,65906.495328,1
...,...,...,...,...,...,...,...,...
101,Swiss International Air Lines,CaptMin,Legacy,Switzerland,90600.0,114.2,120776.204133,1
102,Atlantic Airways,CaptMax,Regional,Iceland,63460.0,83.3,91309.899433,1
103,Atlantic Airways,CaptMin,Regional,Iceland,63460.0,83.3,57853.803700,1
104,Exxaero,CaptMax,Fractional/Corporate,Netherlands,55200.0,68.6,82310.606460,1


# Update salary for status 2

## Load model

In [158]:
# Load the saved model
with open('best_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

## transform dataframe to predict salaries

In [159]:
# Prepare dataframe for model
df_conso_2 = df_conso_2[["Airline", "Top/Base", "Category", "Country", "Income", "Index", "status"]]
df_conso_2_filtered = df_conso_2[df_conso_2["status"] == 2]

# Remove rows with NaN values
df_conso_2_filtered = df_conso_2_filtered.dropna()
df_conso_3_filtered = df_conso_2_filtered

# Drop features
df_conso_2_filtered = df_conso_2_filtered.drop(['Airline', 'Country', 'status'], axis=1)

# Perform one-hot encoding
df_conso_2_filtered = pd.get_dummies(df_conso_2_filtered, columns=['Top/Base', 'Category'])

# Normalize data
scaler = StandardScaler()
columns_to_scale = ['Income', 'Index']
df_conso_2_filtered[columns_to_scale] = scaler.fit_transform(df_conso_2_filtered[columns_to_scale])

## Predict

In [160]:
# Function to make predictions
def make_prediction(row):
    # Convert the row to a 2D array or DataFrame, depending on the model's input requirements
    input_data = row.values.reshape(1, -1)
    # or
    input_data = pd.DataFrame([row])

    return loaded_model.predict(input_data)[0]

# Create a new column with predictions
df_conso_2_filtered["new_salary_USD_GROSS_2023"] = df_conso_2_filtered.apply(make_prediction, axis=1)

# Add back the status column
df_conso_2_filtered["status"] = df_conso_2.loc[df_conso_2_filtered.index, "status"]

# Display the updated dataframe
df_conso_2_filtered

,Income,Index,Top/Base_CaptMax,Top/Base_CaptMin,Category_Cargo,Category_Charter,Category_Fractional/Corporate,Category_Legacy,Category_Major/National/Low Cost,Category_Regional,new_salary_USD_GROSS_2023,status
8,0.053674,0.264734,1,0,0,0,0,0,1,0,162311.441406,2
9,0.053674,0.264734,0,1,0,0,0,0,1,0,103575.245239,2
16,0.570033,0.867686,1,0,0,0,0,0,1,0,158373.626465,2
66,-1.040255,-0.707595,1,0,0,1,0,0,0,0,146655.055664,2
74,-0.512965,-0.495747,1,0,0,1,0,0,0,0,147075.902100,2
...,...,...,...,...,...,...,...,...,...,...,...,...
989,-1.303462,-1.028083,0,1,0,0,1,0,0,0,98938.015503,2
990,0.688521,0.845958,1,0,0,0,0,0,0,1,103174.828857,2
991,0.688521,0.845958,0,1,0,0,0,0,0,1,64254.313477,2
992,0.396456,0.319054,1,0,0,0,1,0,0,0,110000.207520,2


In [161]:
df_conso_3_filtered["new_salary_USD_GROSS_2023"] = df_conso_2_filtered["new_salary_USD_GROSS_2023"]
df_conso_3_filtered = df_conso_3_filtered[["Airline", "Top/Base", "Category", "Country", "Income", "Index", "new_salary_USD_GROSS_2023", "status"]]
merged_df = pd.concat([df_model, df_conso_3_filtered], ignore_index=True)
merged_df

,Airline,Top/Base,Category,Country,Income,Index,new_salary_USD_GROSS_2023,status
0,British Airways Mainline,CaptMax,Legacy,United Kingdom,44480.0,61.5,218679.206632,0
1,British Airways Mainline,CaptMin,Legacy,United Kingdom,44480.0,61.5,105890.008194,0
2,DHL Air UK,CaptMax,Cargo,United Kingdom,44480.0,61.5,207381.790138,0
3,DHL Air UK,CaptMin,Cargo,United Kingdom,44480.0,61.5,146582.638454,0
4,LOT Polish Airlines,CaptMax,Major/National/Low Cost,Poland,16850.0,38.6,65906.495328,1
...,...,...,...,...,...,...,...,...
641,Aeropartner,CaptMin,Fractional/Corporate,Russian Federation,11610.0,40.8,98938.015503,2
642,Airnorth,CaptMax,Regional,Australia,57170.0,75.3,103174.828857,2
643,Airnorth,CaptMin,Regional,Australia,57170.0,75.3,64254.313477,2
644,FlyinGroup,CaptMax,Fractional/Corporate,Belgium,50490.0,65.6,110000.207520,2


In [162]:
merged_df.to_csv("large_db.csv")